# Delta Lake Lab 
## Unit 6: Time Travel

In the previous unit, we-
1. Learned how to change the schema of tables with data in them, and reviewed the impact on files in the data lake and the transaction log

In this unit, we will-
1. Study delta lake's time travel support

### 1. Imports

In [1]:
import pandas as pd

from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession

from delta.tables import *

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [2]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

22/11/01 22:09:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 3. Declare variables

In [3]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  delta-lake-lab


In [4]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  delta-lake-lab


In [5]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  885979867746


In [ ]:
ACCOUNT_NAME = "akhjain"

In [6]:
DATA_LAKE_ROOT_PATH= f"gs://dll-data-bucket-{PROJECT_NUMBER}-{ACCOUNT_NAME}"
DELTA_LAKE_DIR_ROOT = f"{DATA_LAKE_ROOT_PATH}/delta-consumable"
print(DELTA_LAKE_DIR_ROOT)

gs://dll-data-bucket-885979867746/delta-consumable


In [7]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-0bd1b0e2-4659-438c-be86-355bb31b4ac9-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-10887990-f51d-46ad-a768-2f8135d5fd95-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-2517e153-6923-4599-8b2e-5746fcf10973-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-363ee39f-ff75-4e4d-8938-d866d43955ca-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-72b5547b-66aa-4431-8822-5dff110e3c01-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-99635b87-214b-4afa-895e-8c124bf54009-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-9a73bff5-5ed8-4ce2-9383-b6f2865cbc6a-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-b28e7cf4-80c3-4054-8cef-3cff4fe805ea-c000.snappy.par

### 4. History

In [8]:
spark.sql("DESCRIBE HISTORY loan_db.loans_by_state_delta").select("version","timestamp","operation","operationParameters").show(truncate=False)

ANTLR Tool version 4.8 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.8 used for parser compilation does not match the current runtime version 4.9.3
ANTLR Tool version 4.8 used for code generation does not match the current runtime version 4.9.3
ANTLR Runtime version 4.8 used for parser compilation does not match the current runtime version 4.9.3
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used


+-------+-----------------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|version|timestamp              |operation|operationParameters                                                                                                                            |
+-------+-----------------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|11     |2022-11-01 22:07:07.427|WRITE    |{mode -> Append, partitionBy -> []}                                                                                                            |
|10     |2022-11-01 22:06:41.238|WRITE    |{mode -> Append, partitionBy -> []}                                                                                                            |
|9      |2022-11-01 22:06:10.128|WRITE    |{mode -> Append, 

### 5. Lets look at a few versions

In [9]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta VERSION AS OF 1 where addr_state='IA'").show()

+----------+-----+
|addr_state|count|
+----------+-----+
+----------+-----+



In [10]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta VERSION AS OF 10 where addr_state='IA'").show()

+----------+-----+------------------+
|addr_state|count|  collateral_value|
+----------+-----+------------------+
|        IA|  284|2848559.2048476106|
|        IA|  349|3490455.5807275963|
|        IA|  351|3514649.5614278573|
|        IA|  164|1649795.5660065983|
|        IA|   97| 977271.2566990124|
|        IA|   79| 793382.1492048983|
|        IA|  149|1494744.6633344109|
|        IA|    7| 79748.79175558798|
|        IA|  452| 4525824.841281812|
|        IA|  186|1866039.3042891505|
|        IA|  489| 4898122.920044726|
|        IA|  118|1180143.7762261115|
|        IA|  132|1327091.5650634842|
|        IA|  410|4107695.0403600573|
|        IA|   32| 326185.5741464369|
|        IA|    6|62424.697092141556|
|        IA|   12|127582.34045748881|
|        IA|  151|1519729.3072769297|
|        IA|  150|1505116.0433037076|
|        IA|  164|1641533.0867176917|
+----------+-----+------------------+
only showing top 20 rows



In [11]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta VERSION AS OF 5 where addr_state='IA'").show()

+----------+-----+
|addr_state|count|
+----------+-----+
|        IA|  555|
+----------+-----+



### THIS CONCLUDES THIS UNIT. PROCEED TO THE NEXT NOTEBOOK